In [2]:
get_data = []
location = []
data_gu_code = []

In [3]:
year = 2018
gu_list = [
    110,  # 종로구
    215,  # 광진구
    680,  # 강남구
    740,  # 강동구
    500,  # 강서구
    620,  # 관악구
    530,  # 구로구
    545,  # 금천구
    350,  # 노원구
    320,  # 도봉구
    230,  # 동대문구
    590,  # 동작구
    440,  # 마포구
    410,  # 서대문구
    650,  # 서초구
    200,  # 성동구
    290,  # 성북구
    710,  # 송파구
    470,  # 양천구
    560,  # 영등포구
    170,  # 용산구
    380,  # 은평구
    140,  # 중구
    260   # 중랑구
]

In [ ]:
import requests
import xml.etree.ElementTree as ET
import json

data_find_test = None

for guGun in gu_list : 
    # URL과 요청 파라미터 설정   
    
    url = "http://apis.data.go.kr/B552061/frequentzoneFreezing/getRestFrequentzoneFreezing?"
    api_key = 'null'
    
    params = { 
        "serviceKey": api_key,
        "searchYearCd": year,  # 해당 년도의 데이터 요청
        "siDo": "11",  # 서울시
        "guGun": guGun,  #  
        "type": "XML",  # 응답 타입 XML
        "numOfRows": "100",  # 한 페이지에 출력될 데이터 개수
        "pageNo": "1"  # 페이지 번호
    }
    
    # API 요청
    response = requests.get(url, params=params)
    

    try : 
            # XML 파싱
        root = ET.fromstring(response.text)
        
        # geom_json 값 추출
        geom_json_str = root.find(".//geom_json").text
        temp_location = root.find(".//sido_sgg_nm").text
        temp_location_code = root.find(".//spot_cd").text
        
        # geom_json 파싱
        geom_data = json.loads(geom_json_str)
        
        # coordinates 값 추출
        coordinates = geom_data["coordinates"]
        
        # 중첩된 리스트를 평탄화 (1차원 리스트로 변환)
        flat_coordinates = [coord for sublist in coordinates for coord in sublist]
        
        # 데이터를 저장
        if temp_location not in location : 
            get_data.append(flat_coordinates)
            location.append(temp_location)
            data_gu_code.append(temp_location_code) 
            data_find_test = True 
        print(f'{year}년도 {guGun}는 성공!!')  
    except : 
        print(f'{year}년도 {guGun}는 Null') 

2018년도 110는 Null
<response>
    <header>
        <resultCode>00</resultCode>
        <resultMsg>NORMAL_CODE</resultMsg>
    </header>
    <body>
        <items>
            <item>
                <afos_fid>6571253</afos_fid>
                <afos_id>2019079</afos_id>
                <bjd_cd>1121510700</bjd_cd>
                <spot_cd>11215001</spot_cd>
                <sido_sgg_nm>서울특별시 광진구1</sido_sgg_nm>
                <spot_nm>서울특별시 광진구 화양동(건대입구역 부근)</spot_nm>
                <occrrnc_cnt>3</occrrnc_cnt>
                <caslt_cnt>7</caslt_cnt>
                <dth_dnv_cnt>0</dth_dnv_cnt>
                <se_dnv_cnt>2</se_dnv_cnt>
                <sl_dnv_cnt>4</sl_dnv_cnt>
                <wnd_dnv_cnt>1</wnd_dnv_cnt>
                <geom_json>{"type":"Polygon","coordinates":[[[127.07292728,37.54129396],[127.07289276,37.54101603],[127.07279052,37.54074879],[127.0726245,37.5405025],[127.07240106,37.54028662],[127.07212881,37.54010946],[127.07181819,37.53997781],[127.07148116,37.5398

In [5]:
import pandas as pd

In [6]:
if(data_find_test) :
    data = pd.DataFrame({'지역' : location, '법정동코드' : data_gu_code, '사고위치' : get_data})
    data.index = data.index + 1
    file = f'{year}_seoul_icy_road_accident.csv'
    data.to_csv(file)
    print(f'{file} 파일 생성 성공')
    data_find_test = False

2018_seoul_icy_road_accident.csv 파일 생성 성공
